In [1]:
import regex as re
import pandas as pd

In [2]:
f=open('./kafka_10k.log')
lines=f.readlines()

In [3]:
split_pt=re.compile(r'([,\s$@<>\[\]\'"=]|\((?!\))|(?<!\()\)|[:\.](?=\s)|(?<=\b[a-zA-Z]+):|:(?=[a-zA-Z]+\b)|:(?=\/[a-zA-Z0-9]))')
hm_pt=re.compile(r'(\[.*?\]\s[A-Z]+)(.*)')

In [4]:
header=[]
msg=[]
for line in lines:
    h,m=hm_pt.match(line).groups()
    
    header.append(h)
    msg.append(m)

In [5]:
print(header[0])
print(msg[0])

[2020-03-11 10:11:07,562] INFO
 Registered kafka:type=kafka.Log4jController MBean (kafka.utils.Log4jControllerRegistration$)


In [6]:
header_tokens=[]
for h in header:
    tokens=[token for token in re.split(r'([\s$@<>\[\]\'"=]|\((?!\))|(?<!\()\)|[:\.,](?=\s)|(?<=\b[a-zA-Z]+):|:(?=[a-zA-Z]+\b))',h) if token!='' and token!=None]
    header_tokens.append(tokens)

In [7]:
msg_tokens=[]
for m in msg:
    tokens=[token for token in split_pt.split(m) if token!='' and token!=None]
    msg_tokens.append(tokens)

In [8]:
pd.DataFrame({'header_tokens':header_tokens,'msg_tokens':msg_tokens}).to_csv('tokens.csv')

In [9]:
header_anno=[]
for tokens in header_tokens:
    anno=['O']*len(tokens)
    anno[1],anno[3],anno[6]='B-DATE','B-TIME','B-LVL'

    header_anno.append(anno)

In [10]:
list(zip(header_tokens[0],header_anno[0]))

[('[', 'O'),
 ('2020-03-11', 'B-DATE'),
 (' ', 'O'),
 ('10:11:07,562', 'B-TIME'),
 (']', 'O'),
 (' ', 'O'),
 ('INFO', 'B-LVL')]

In [11]:
msg_anno=[]
for tokens in msg_tokens:
    anno=['<O>']*len(tokens)
    msg_anno.append(anno)

In [12]:
re_date=r'\d{2}\/\d{2}\/\d{4}'
re_time=r'^\d{2}:\d{2}$'

re_obj=r'[a-zA-Z]+-\d{1,3}$'
re_cls=r'(\.([A-Z]+[a-z]*)+)|([A-Z]+[a-z]+){2,}$'
re_func=r'\S*\(\)$'
re_path=r'^\'?\/[a-zA-Z]'
re_ip=r'(?:[0-9]{1,3}\.){3}[0-9]{1,3}'

re_host=r':\d{3,5}$'
re_url=r'[a-z]+:\/\/\S*'


In [13]:
def sub_tag(ntokens,ntags, patt_tag_list):

    def _sub_tag(ntokens,ntags, patt, target_tag):
        for tokens,tags in zip(ntokens,ntags):
            for i in range(len(tokens)):
                if re.search(patt,tokens[i]):
                    tags[i]=target_tag

    for patt,tag in patt_tag_list:
        _sub_tag(ntokens,ntags,patt,tag)

In [14]:
sub_tag(msg_tokens,msg_anno,
    [(re_date,'B-DATE'),
    (re_time,'B-TIME'),
    (re_cls,'B-CLS'),
    (re_func,'B-FUNC'),
    (re_path,'B-PATH'),
    (re_ip,'B-IP'),
    (re_host,'B-HOST'),
    (re_url,'B-URL')])

In [15]:
list(zip(msg_tokens[0],msg_anno[0]))

[(' ', '<O>'),
 ('Registered', '<O>'),
 (' ', '<O>'),
 ('kafka', '<O>'),
 (':', '<O>'),
 ('type', '<O>'),
 ('=', '<O>'),
 ('kafka.Log4jController', 'B-CLS'),
 (' ', '<O>'),
 ('MBean', '<O>'),
 (' ', '<O>'),
 ('(', '<O>'),
 ('kafka.utils.Log4jControllerRegistration', 'B-CLS'),
 ('$', '<O>'),
 (')', '<O>')]

In [16]:
with open('./header_check.txt','w') as f:
    for tokens,tags in zip(header_tokens,header_anno):
        for token,tag in zip(tokens,tags):
            f.write(token+' '+tag+'\n')
        f.write('\n')

In [17]:
with open('./msg_check.txt','w') as f:
    for tokens,tags in zip(msg_tokens,msg_anno):
        for token,tag in zip(tokens,tags):
            f.write(token+' '+tag+'\n')
        f.write('\n')

In [18]:
df=pd.DataFrame({
    'tokens':[h+m for h,m in zip(header_tokens,msg_tokens)],
    'tags':[h+m for h,m in zip(header_anno,msg_anno)],
    'raw_log':[h+m for h,m in zip(header,msg)]
    })
df.to_csv('./kafka_ds.csv')